In [1]:
import ODM

In [2]:
ODM.initApp(ext_globals=globals())

Clases instanciadas


In [3]:
#1
for doc in Compra.aggregate([{"$match": { "cliente":"Pedro Sánchez"}}]):
    print(doc)

{'_id': ObjectId('671cdeef605dd8c94a71ddd4'), 'compra_id': 'comp2', 'cliente': 'Pedro Sánchez', 'fecha_compra': datetime.datetime(2024, 10, 17, 0, 0), 'productos': [{'producto_id': 'Pantalón vaquero', 'cantidad': 2, 'precio_unitario': 49.99}], 'monto_total': 99.98, 'almacen_id': 'alm2', 'direccion': 'Birmingham City Centre, Birmingham', 'location': {'type': 'Point', 'coordinates': [-1.909386, 52.475576]}}
{'_id': ObjectId('671cdef0605dd8c94a71ddd5'), 'compra_id': 'comp3', 'cliente': 'Pedro Sánchez', 'fecha_compra': datetime.datetime(2024, 10, 18, 0, 0), 'productos': [{'producto_id': 'Vestido de fiesta', 'cantidad': 1, 'precio_unitario': 89.99}], 'monto_total': 89.99, 'almacen_id': 'alm3', 'direccion': 'Market Street, Manchester', 'location': {'type': 'Point', 'coordinates': [-2.24474, 53.483218]}}


In [4]:
#2
for doc in Producto.aggregate([{"$match": { "nombre": "Blusa de seda"}}, { "$project": { "_id": 0, "proveedor":1 }}]):
    print(doc)

{'proveedor': ['Modas Paqui', 'Textiles García', 'Elegancia S.L.']}


In [5]:
#3
for doc in Compra.aggregate([{ "$group": {"_id":"$cliente", "totalProductos": { "$push": "$productos" }}}, {"$match": {"_id":"Isabel Moreno"}}, { "$project": {"_id":0}}]):
    print(doc)

{'totalProductos': [[{'producto_id': 'Reloj de pulsera', 'cantidad': 1, 'precio_unitario': 129.99}], [{'producto_id': 'Bolso de mano', 'cantidad': 1, 'precio_unitario': 45.99}, {'producto_id': 'Camiseta manga corta', 'cantidad': 3, 'precio_unitario': 19.99}]]}


In [6]:
#4
for doc in Producto.aggregate([ {"$match": { "proveedor": { "$elemMatch": { "$in": ["Modas Paqui"]}}, "nombre":{ "$regex": "manga corta" }}}, {"$project": {"nombre":1, "_id":0}}]):
    print(doc)

{'nombre': 'Camiseta manga corta'}
{'nombre': 'Camiseta manga corta básica'}
{'nombre': 'Camiseta manga corta deportiva'}
{'nombre': 'Camiseta manga corta elegante'}
{'nombre': 'Camiseta manga corta estampada'}


In [7]:
#5
for doc in Compra.aggregate([ { "$match": { "cliente": 'Javier López'}},{"$lookup": { "from":"Producto", "localField": "productos.producto_id", "foreignField":"nombre", "as":"productos"}},{"$unwind":"$productos"},{ "$group": { "_id": 'Javier López', "pesoTotal": { "$sum":"$productos.peso" }, "volumenTotal": { "$sum": "$productos.volumen"}}}]):
    print(doc)

{'_id': 'Javier López', 'pesoTotal': 0.9, 'volumenTotal': 0.0045000000000000005}


In [8]:
#6
for doc in Envio.aggregate([{"$group":{"_id":{"almacen":"$almacen","año":{"$year":"$fecha_envio"},"mes":{"$month":"$fecha_envio"}},"totalEnvios":{"$sum":1}}},{"$group":{"_id":{"almacen":"$_id.almacen","mes":"$_id.mes"},"promedioEnviosPorMes":{"$avg":"$totalEnvios"}}},{"$project":{"_id":1,"promedioEnviosPorMes":{"$round":["$promedioEnviosPorMes",2]}}}]):
    print(doc)

{'_id': {'almacen': 'Almacén Bilbao', 'mes': 6}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Principal Barcelona', 'mes': 12}, 'promedioEnviosPorMes': 2.0}
{'_id': {'almacen': 'Almacén Sevilla', 'mes': 10}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Málaga', 'mes': 11}, 'promedioEnviosPorMes': 2.0}
{'_id': {'almacen': 'Almacén Principal Barcelona', 'mes': 10}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Sevilla', 'mes': 12}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Málaga', 'mes': 12}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Principal Barcelona', 'mes': 8}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Central Madrid', 'mes': 11}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Central Madrid', 'mes': 9}, 'promedioEnviosPorMes': 2.0}
{'_id': {'almacen': 'Almacén Sevilla', 'mes': 11}, 'promedioEnviosPorMes': 1.0}
{'_id': {'almacen': 'Almacén Valencia', 'mes': 7}, 'promedioEnviosPorMes': 1

In [9]:
#7

for doc in Proveedor.aggregate([{"$group": {"_id": "$nombre","totalFacturacion": { "$sum": "$facturacion" }}},{"$sort": { "totalFacturacion": -1 }},{"$limit": 3},{"$project": {"proveedor": "$_id","volumenfacturacion": "$totalFacturacion","_id": 0}}]):
    print(doc)

{'proveedor': 'Elegancia S.L.', 'volumenfacturacion': 320000}
{'proveedor': 'Textiles García', 'volumenfacturacion': 280000}
{'proveedor': 'Tejidos del Sur', 'volumenfacturacion': 260000}


In [10]:
#8
for doc in Almacen.aggregate([{"$geoNear": {"near": { "type": "Point", "coordinates": [-0.128049, 51.508037] }, "distanceField": "distancia","maxDistance": 100000, "spherical": "true"}},{"$project": {"_id": 1,"nombre": 1,"direccion": 1,"distancia": 1,"location": 1}},{"$sort": { "distancia": 1 }}]):
    print(doc)

{'_id': ObjectId('671cdef9605dd8c94a71ddf3'), 'nombre': 'Almacén Central Madrid', 'direccion': 'Trafalgar Square, London', 'location': {'type': 'Point', 'coordinates': [-0.128049, 51.508037]}, 'distancia': 0.0}
{'_id': ObjectId('671cdefe605dd8c94a71ddf7'), 'nombre': 'Almacén Bilbao', 'direccion': 'Big Ben, Westminster, London', 'location': {'type': 'Point', 'coordinates': [-0.124572, 51.500704]}, 'distancia': 851.1123154600872}
{'_id': ObjectId('671cdefb605dd8c94a71ddf5'), 'nombre': 'Almacén Valencia', 'direccion': 'St. Paul’s Cathedral, London', 'location': {'type': 'Point', 'coordinates': [-0.098451, 51.513787]}, 'distancia': 2148.1580747695166}
{'_id': ObjectId('671cdf04605dd8c94a71ddfc'), 'nombre': 'Almacén Las Palmas', 'direccion': 'Hyde Park, London', 'location': {'type': 'Point', 'coordinates': [-0.162207, 51.507489]}, 'distancia': 2367.450606121052}
{'_id': ObjectId('671cdefa605dd8c94a71ddf4'), 'nombre': 'Almacén Principal Barcelona', 'direccion': 'The Shard, 32 London Bridge S

In [11]:
#9
for doc in Compra.aggregate([{"$match": {"location": {"$geoWithin": {"$geometry": {"type": "Polygon","coordinates": [[[-5.0, 50.0],[-5.0, 59.0],[2.0, 59.0],[2.0, 50.0],[-5.0, 50.0]]]}}}}},{"$project": {"_id":0,"compra_id": 1,"direccion": 1}}]):
    print(doc)

{'compra_id': 'comp2', 'direccion': 'Birmingham City Centre, Birmingham'}
{'compra_id': 'comp1', 'direccion': 'Trafalgar Square, London'}
{'compra_id': 'comp8', 'direccion': 'Leicester Square, London'}
{'compra_id': 'comp7', 'direccion': 'Coventry Street, London'}
{'compra_id': 'comp6', 'direccion': 'Oxford Street, London'}
{'compra_id': 'comp5', 'direccion': 'Bold Street, Liverpool'}
{'compra_id': 'comp3', 'direccion': 'Market Street, Manchester'}
{'compra_id': 'comp4', 'direccion': 'Royal Mile, Edinburgh'}


In [12]:
#10 En este caso la query no va a mostrar nada ya que lo almacena en una nueva colección que se va a llamar "Envios_AlmacenSevilla"
for doc in Envio.aggregate([{"$match": {"fecha_envio": "ISODate('2024-12-22T00:00:00.000Z')", "almacen": "Almacén Principal Barcelona"}}, { "$out": "Envios_AlmacenSevilla"}]):
    print(doc)
